In [1]:
import pandas as pd

In [5]:
sheet_id = "1fS0JePwvPNzeL6alOVgN1Dw_458GqQ_QMkx3qaRqLGU"
url = "https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [29]:
base = pd.read_csv(url.format(sheet_id=sheet_id, sheet_name="base")).iloc[:, :3]
annot_a = pd.read_csv(url.format(sheet_id=sheet_id, sheet_name="annot_andrea"))
annot_b = pd.read_csv(url.format(sheet_id=sheet_id, sheet_name="annot_amy"))

In [34]:
def clean_dfr(dfr, suffix=""):
    dfr = dfr.iloc[:, :4]
    dfr.columns=["Pattern", "rating_"+suffix, "hascon_"+suffix, "con_"+suffix]
    dfr.iloc[:,1] = dfr.iloc[:,1].fillna(-1)
    dfr.iloc[:,3] = dfr.iloc[:,3].fillna("")
    return dfr 

In [43]:
annot_a = clean_dfr(annot_a, "a")
annot_b.iloc[:, 1] = pd.to_numeric(annot_b.iloc[:, 1].str.replace("x", "nan"), errors='coerce')
annot_b = clean_dfr(annot_b, "b")

In [45]:
m_dfr = base.merge(annot_a, on="Pattern").merge(annot_b, on="Pattern")

In [46]:
m_dfr

,Pattern,model score,weighted score,rating_a,hascon_a,con_a,rating_b,hascon_b,con_b
0,推一個中華電信,7.8706,0.2189,1.0,1,推一個X,1.0,1,推一個N
1,跟中華,0.8342,0.1825,-1.0,0,,-1.0,0,
2,推薦中華,7.2743,0.1209,1.0,0,,1.0,0,
3,便宜很多冷門片,1.1679,0.1046,1.0,0,,1.0,0,
4,599價格,0.6231,0.0905,-1.0,0,,-1.0,0,
...,...,...,...,...,...,...,...,...,...
243,台哥大很爛,-6.8698,-0.1157,0.0,0,,0.0,0,
244,然後五樓蟲真的,-0.6196,-0.1208,-1.0,0,,-1.0,0,
245,學校裡面,-2.0681,-0.1208,-1.0,0,,-1.0,0,
246,中華電信的,-1.3989,-0.1210,-1.0,0,,-1.0,0,


In [48]:
pd.crosstab(m_dfr.rating_a, m_dfr.rating_b)

rating_b,-1.0,0.0,1.0
rating_a,,,
-1.0,109,17,8
0.0,4,88,0
1.0,0,0,22


In [49]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(m_dfr.rating_a, m_dfr.rating_b)

0.8004550246934132

In [56]:
((m_dfr.rating_a != -1) | (m_dfr.rating_b != -1)).sum()

139

In [57]:
m_dfr.shape

(248, 9)

In [58]:
110/248

0.4435483870967742

In [65]:
(m_dfr.hascon_a | m_dfr.hascon_b).sum()

55

In [60]:
m_dfr.to_csv("../../../data/caprice/cons_annotated.csv", index=None)